In [1]:
!pwd

/infodev1/infoderm/Projects/Naresh/scripts/misc/TCGA_BRCA_PAM50


In [2]:
# for loading/processing the images  
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.preprocessing.image import img_to_array 
from tensorflow.keras.applications.vgg16 import preprocess_input 

# models 
from tensorflow.keras.applications.vgg16 import VGG16 
from tensorflow.keras.models import Model

# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle# for loading/processing the 
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import glob
import cv2 as cv
from matplotlib import pyplot as plt
from __future__ import absolute_import, division, print_function, unicode_literals

import argparse
import logging
import os
import sys
import tensorflow as tf
import matplotlib.pylab as plt
from tensorflow.keras import layers
from tensorflow.keras import models
import numpy as np
import PIL.Image as Image
import glob
#from preprocess import Preprocess, format_example, format_example_tf, update_status


2023-07-19 19:32:40.424208: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
import autogluon.core as ag
#from autogluon.vision import ImagePredictor, ImageDataset

In [460]:
#flowers_full = glob.glob("/research/bsi/projects/breast/s301449.LARdl/processing/MOLI/MOLI_code/Keras/xiaojia_code/methyl_data/AD_vs_CN.images1/*/*/*png")
seed="1010"
flowers_full = glob.glob("/infodev1/infoderm/Projects/Naresh/scripts/misc/TCGA_BRCA_PAM50/images/images."+seed+"/*/*/*png")
flowers = {}
for i in flowers_full:
    flowers[os.path.basename(i)] = i

In [461]:
#len(flowers_full),flowers_full[0],

In [462]:
keys = list(flowers.keys())
keys[1:5],flowers[keys[1]]


(['TCGA.B6.A0RT.01A.png',
  'TCGA.BH.A0AV.01A.png',
  'TCGA.LL.A6FR.01A.png',
  'TCGA.AO.A12F.01A.png'],
 '/infodev1/infoderm/Projects/Naresh/scripts/misc/TCGA_BRCA_PAM50/images/images.1010/test/2/TCGA.B6.A0RT.01A.png')

In [463]:
cate={}
lbl={}
for i in keys:
    lbl[i]=os.path.basename(os.path.dirname(flowers[i]))
    cate[i]=os.path.basename(os.path.dirname(os.path.dirname(flowers[i])))

In [464]:
#img = load_img(flowers[keys[10]], target_size=(224,224))
#img.save("temp.png")
#img = np.array(img) 
#plt.figure(num=None, figsize=(8, 6), dpi=80)
#plt.imshow(img)

In [465]:
#img = cv.imread(flowers[keys[10]], cv.IMREAD_UNCHANGED)#cv2.IMREAD_COLOR)IMREAD_UNCHANGED
#img = cv.resize(img, (224,224), interpolation = cv.INTER_AREA)
#cv.imwrite("cv2_tmp.png", img)
#plt.figure(num=None, figsize=(8, 6), dpi=80)
#plt.imshow(img)

In [466]:
from PIL import Image
#image = Image.open(flowers[keys[10]])
#image = image.resize((224,224),Image.ANTIALIAS)
#image.save(fp="newimage.png")

In [467]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from skimage.color import rgb2hsv, rgb2gray, rgb2yuv
from skimage import color, exposure, transform
from skimage.exposure import equalize_hist

In [468]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array
    #img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    #img = np.array(img)
    img = cv.imread(file, cv.IMREAD_UNCHANGED)#cv2.IMREAD_COLOR)IMREAD_UNCHANGED
    img = cv.resize(img, (224,224), interpolation = cv.INTER_CUBIC)
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [469]:
data = {}
#p = "/research/bsi/projects/breast/s301449.LARdl/processing/MOLI/MOLI_code/Keras/xiaojia_code/output/all_vec/allvec"
p = "/infodev1/infoderm/Projects/Naresh/scripts/Kiosk/runs/cats_dogs_feavec"
# lop through each image in the dataset
for flower in keys:
    # try to extract the features and update the dictionary
    #try:
    #print(flower)
    feat = extract_features(flowers[flower],model)
    data[flower] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    #except:
    #    with open(p,'wb') as file:
    #        pickle.dump(data,file)

In [470]:
len(data)

828

In [471]:
data[keys[0]].shape

(1, 4096)

In [472]:
flowers[keys[0]]

'/infodev1/infoderm/Projects/Naresh/scripts/misc/TCGA_BRCA_PAM50/images/images.1010/test/2/TCGA.LL.A8F5.01A.png'

In [473]:
# get a list of the filenames
#filenames = np.array(list(data.keys()))
filenames = np.array([i for i in keys])
# get a list of just the features
#feat = np.array(list(data.values()))
feat = np.array([data[i] for i in keys])
# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)


In [474]:
feat.shape,filenames[:5],keys[1:5],len(filenames)

((828, 4096),
 array(['TCGA.LL.A8F5.01A.png', 'TCGA.B6.A0RT.01A.png',
        'TCGA.BH.A0AV.01A.png', 'TCGA.LL.A6FR.01A.png',
        'TCGA.AO.A12F.01A.png'], dtype='<U20'),
 ['TCGA.B6.A0RT.01A.png',
  'TCGA.BH.A0AV.01A.png',
  'TCGA.LL.A6FR.01A.png',
  'TCGA.AO.A12F.01A.png'],
 828)

In [475]:
list_lbl = [lbl[i] for i in keys]
list_cate = [cate[i] for i in keys]
list_lbl_cate = [lbl[i]+'_'+cate[i] for i in keys]

In [476]:
from collections import Counter
#list_lbl[0:5],list_cate[0:5],list_lbl_cate[0:5]
Counter(list_lbl_cate)

Counter({'2_test': 22,
         '3_test': 10,
         '0_test': 64,
         '1_test': 25,
         '2_train': 109,
         '3_train': 53,
         '0_train': 302,
         '1_train': 122,
         '2_val': 22,
         '3_val': 10,
         '0_val': 64,
         '1_val': 25})

In [477]:
cate_train = [list_lbl[i] for i in range(0,len(list_cate),1) if list_cate[i] == "train" ]
cate_val = [list_lbl[i] for i in range(0,len(list_cate),1) if list_cate[i] == "val" ]
cate_test = [list_lbl[i] for i in range(0,len(list_cate),1) if list_cate[i] == "test" ]

In [478]:
train_ind = [i for i in range(0,len(list_cate),1) if list_cate[i] == "train" ]
val_ind = [i for i in range(0,len(list_cate),1) if list_cate[i] == "val" ]
test_ind = [i for i in range(0,len(list_cate),1) if list_cate[i] == "test" ]

In [479]:
len(train_ind),len(val_ind),len(test_ind)

(586, 121, 121)

In [480]:
val_np = feat[val_ind,]
filenm_val = [ filenames[i] for i in val_ind]
test_np = feat[test_ind,]
filenm_test = [ filenames[i] for i in test_ind]
train_np = feat[train_ind,]
filenm_train = [ filenames[i] for i in train_ind]

In [481]:
#l_lst
#len(cate_0),len(cate_1)
#val_np.shape,test_np.shape,train_np.shape,filenames[test_ind[0]]

In [482]:
pca = PCA(n_components=50, random_state=22)
pca.fit(train_np)
x = pca.transform(train_np)
x_val = pca.transform(val_np)
x_test = pca.transform(test_np)

#range_n_clusters = [2,3,4,5,6,7,8,9]
#silhouette_avg = []
#for num_clusters in range_n_clusters:
# 
# # initialise kmeans
# kmeans = KMeans(n_clusters=num_clusters)
# kmeans.fit(x)
# cluster_labels = kmeans.labels_
# silhouette_avg.append(silhouette_score(x, cluster_labels))

In [483]:
#filenm_train[0:5],set(cate_train)

In [484]:
y_ori_train = [ i for i in cate_train ]
y_ori_val = [ i for i in cate_val ]
y_ori_test = [ i for i in cate_test ]


In [485]:
len(y_ori_train),len(y_ori_val),len(y_ori_test),x.shape,x_val.shape,x_test.shape

(586, 121, 121, (586, 50), (121, 50), (121, 50))

In [486]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(x, y_ori_train)
y_pred = classifier.predict(x_val)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_ori_val, y_pred))

[[59  5  0  0]
 [17  6  2  0]
 [ 4  2 15  1]
 [ 8  0  0  2]]


In [487]:
print(classification_report(y_ori_val, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.92      0.78        64
           1       0.46      0.24      0.32        25
           2       0.88      0.68      0.77        22
           3       0.67      0.20      0.31        10

    accuracy                           0.68       121
   macro avg       0.67      0.51      0.54       121
weighted avg       0.67      0.68      0.64       121



In [488]:
y_pred = classifier.predict(x_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_ori_test, y_pred))
print(classification_report(y_ori_test, y_pred))

[[57  5  0  2]
 [17  5  3  0]
 [ 7  0 15  0]
 [ 8  0  1  1]]
              precision    recall  f1-score   support

           0       0.64      0.89      0.75        64
           1       0.50      0.20      0.29        25
           2       0.79      0.68      0.73        22
           3       0.33      0.10      0.15        10

    accuracy                           0.64       121
   macro avg       0.57      0.47      0.48       121
weighted avg       0.61      0.64      0.60       121



In [489]:
y_pred_train = classifier.predict(x)
print(classification_report(y_ori_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.70      0.93      0.80       302
           1       0.61      0.38      0.46       122
           2       0.91      0.77      0.84       109
           3       0.79      0.28      0.42        53

    accuracy                           0.73       586
   macro avg       0.75      0.59      0.63       586
weighted avg       0.73      0.73      0.70       586



In [490]:
#fobj =open('out.xls','wt')
#for i in  range(0,len(filenm_val),1):
#    fobj.write(flowers[filenm_val[i]]+" "+y_ori_val[i]+" "+y_pred[i]+"\n")
#for i in  range(0,len(filenm_train),1):
#    fobj.write(flowers[filenm_train[i]]+" "+y_ori_train[i]+" "+y_pred_train[i]+"\n")
#fobj.close()

In [491]:
import autogluon as ag
from autogluon.tabular import TabularPredictor as task

In [492]:
x.shape,x_val.shape,x_test.shape

((586, 50), (121, 50), (121, 50))

In [493]:
colnames=[ "feature"+str(i) for i in range(0,50,1)]
len(colnames)

50

In [494]:
train_df = pd.DataFrame(x, columns = colnames)
test_df = pd.DataFrame(x_test, columns = colnames)
val_df = pd.DataFrame(x_val, columns = colnames)

In [495]:
#train_df.shape,test_df.shape,val_df.shape

In [496]:
train_df['label']=y_ori_train
test_df['label']=y_ori_test
val_df['label']=y_ori_val

In [497]:
save_path="/infodev1/infoderm/Projects/Naresh/scripts/Kiosk/runs/tmp_ag"
predictor = task(label='label', path=save_path,eval_metric = "balanced_accuracy").fit(train_data=train_df,tuning_data=val_df)

Beginning AutoGluon training ...
AutoGluon will save models to "/infodev1/infoderm/Projects/Naresh/scripts/Kiosk/runs/tmp_ag/"
AutoGluon Version:  0.7.0
Python Version:     3.9.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Tue Jun 20 11:48:01 UTC 2023
Train Data Rows:    586
Train Data Columns: 50
Tuning Data Rows:    121
Tuning Data Columns: 50
Label Column: label
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	4 unique label values:  ['2', '3', '0', '1']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 4
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    391328.84 MB
	Train Data (Original)  Memory Usage: 0.14 MB (0.0%

In [498]:
#predictor = task.fit(train_data=train_df, tuning_data=val_df, label='label',eval_metric = "accuracy")
import autogluon.core as ag

#gbm_options = {  # specifies non-default hyperparameter values for lightGBM gradient boosted trees
#    'num_boost_round': 100,  # number of boosting rounds (controls training time of GBM models)
#    'num_leaves': ag.space.Int(lower=3, upper=50, default=36),  # number of leaves in trees (integer hyperparameter)
#}

#hyperparameters = {  # hyperparameters of each model type
#                   'GBM': gbm_options,
#                    # NOTE: comment this line out if you get errors on Mac OSX
#                  }  # When these keys are missing from hyperparameters dict, no models of that type are trained

#time_limit = 2*60  # train various models for ~2 min
#num_trials = 5  # try at most 5 different hyperparameter configurations for each type of model
#search_strategy = 'auto'  # to tune hyperparameters using random search routine with a local scheduler

#hyperparameter_tune_kwargs = {  # HPO is not performed unless hyperparameter_tune_kwargs is specified
#    'num_trials': num_trials,
#    'scheduler' : 'local',
#    'searcher': search_strategy,
#}

#save_path="/research/bsi/projects/breast/s301449.LARdl/processing/MOLI/MOLI_code/Keras/xiaojia_code/Kevin_Pac/ag_model"
#predictor = task(label='label', path=save_path,eval_metric = "accuracy").fit(train_data=train_df,tuning_data=val_df,  time_limit=time_limit,
#    hyperparameters=hyperparameters, hyperparameter_tune_kwargs=hyperparameter_tune_kwargs)

In [499]:
performance = predictor.evaluate(test_df)

Evaluation: balanced_accuracy on test data: 0.6235653409090909
Evaluations on test data:
{
    "balanced_accuracy": 0.6235653409090909,
    "accuracy": 0.743801652892562,
    "mcc": 0.5853273109717716
}


In [500]:
#predictor.fit_summary()

In [501]:
#performance = predictor.evaluate(val_df)

In [502]:
y_test = test_df['label']

In [503]:
test_data_nolab = test_df.drop(columns=['label'])
y_pred = predictor.predict(test_data_nolab)
#print("Predictions:  \n", y_pred)
#perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

In [504]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import glob
import numpy as np
from pandas import read_csv
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn import metrics
import re
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
def report(fish_list, result_list,prob, name):
    tn, fp, fn, tp = metrics.confusion_matrix(fish_list, result_list).ravel()
    tn=int(tn)
    fp=int(fp)
    fn=int(fn)
    tp=int(tp)
    #sys.exit(0)    
    sensitivity=round(tp/(tp+fn),2)
    #print(sensitivity)
    #sys.exit(0)    
    specificity=round(tn/(tn+fp),2)
    accuracy=round((tp + tn)/(tn + fp + fn + tp),2)
    fpr, tpr, thresholds = metrics.roc_curve(fish_list, result_list, pos_label=1)
    #auc = round(metrics.auc(fpr, tpr),2)
    #print(name , "TN:",tn, "FP:",fp, "FN:",fn, "TP:",tp, "Sensitivity:",sensitivity, "Specificity:",specificity, "AUC:",auc,"Accuracy:",accuracy)
    #print(name,tn,fp,fn,tp,sensitivity,specificity,auc,accuracy)
    target_names = ['class 0', 'class 1']
    cr = classification_report(fish_list, result_list, target_names=target_names)
    cr = re.sub('\n+', '\n', cr)
    cr = re.sub(' +', ' ', cr)
    cr = re.sub('\n ', '\n', cr)
    cr_a = cr.split("\n")
    pres = []
    recall = []
    f1 = []
    for i in range(1,3,1):
        cr_a_t = cr_a[i].split(" ")
        pres.append(cr_a_t[len(cr_a_t)-4])
        recall.append(cr_a_t[len(cr_a_t)-3])
        f1.append(cr_a_t[len(cr_a_t)-2])
    presc = ' '.join(pres)
    recallc = ' '.join(recall)
    f1c = ' '.join(recall)
    weighted_acc=cr_a[5].split(" ")[2]
    
    #test_precision, test_recall, _ = precision_recall_curve(fish_list, prob)
    #auc1 =  auc(test_recall, test_precision)
    fpr, tpr, thresholds = metrics.roc_curve(fish_list, prob)
    auc1 = round(metrics.auc(fpr, tpr),2)
    print(name,tn,fp,fn,tp,sensitivity,specificity,auc1,accuracy,presc,recallc,f1c,weighted_acc)
    #print(cr_a)

In [505]:
#predictor.leaderboard(test_df,silent=True)

In [506]:
#print("name","tn","fp","fn","tp","sensitivity","specificity","auc","accuracy","presc1","presc2","recallc1","recallc2","f1_score_c1","f1_score_c2","weighted_acc")  
myfile = open(seed+"_all.txt", mode='wt')
y_pred = predictor.predict(test_df, model='NeuralNetTorch')
y_prob = predictor.predict_proba(test_df)
y_pred=[int(i) for i in list(y_pred)]
y_true=[int(i) for i in list(test_df["label"])]
#report(y_true, y_pred,list(y_prob.iloc[:,1]), "test")
for i in range(0,len(y_true)):
    inx_mx=np.argmax(y_prob.iloc[i,:])
    inx_mx_val=y_prob.iloc[i,inx_mx]
    myfile.write("autogluon"+" "+str(i)+" "+str(y_true[i])+" "+"test"+" "+str(inx_mx_val)+" "+str(inx_mx)+"\n")
y_pred = predictor.predict(val_df, model='NeuralNetTorch')
y_prob = predictor.predict_proba(val_df)
y_pred=[int(i) for i in list(y_pred)]
y_true=[int(i) for i in list(val_df["label"])]
#report(y_true, y_pred,list(y_prob.iloc[:,1]), "val")
for i in range(0,len(y_true)):
    inx_mx=np.argmax(y_prob.iloc[i,:])
    inx_mx_val=y_prob.iloc[i,inx_mx]
    myfile.write("autogluon"+" "+str(i)+" "+str(y_true[i])+" "+"val"+" "+str(inx_mx_val)+" "+str(inx_mx)+"\n")
y_pred = predictor.predict(train_df, model='NeuralNetTorch')
y_prob = predictor.predict_proba(train_df)
y_pred=[int(i) for i in list(y_pred)]
y_true=[int(i) for i in list(train_df["label"])]
#report(y_true, y_pred,list(y_prob.iloc[:,1]), "train")
for i in range(0,len(y_true)):
    inx_mx=np.argmax(y_prob.iloc[i,:])
    inx_mx_val=y_prob.iloc[i,inx_mx]
    myfile.write("autogluon"+" "+str(i)+" "+str(y_true[i])+" "+"train"+" "+str(inx_mx_val)+" "+str(inx_mx)+"\n")
myfile.close()